In [1]:
import os
import json
import pandas as pd
import zipfile
import cv2
import numpy as np
import tensorflow as tf
from google.colab import files
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Upload kaggle.json
files.upload()

# Make directory to store kaggle.json
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

# Set permission
!chmod 600 ~/.kaggle/kaggle.json

# Install Kaggle API
!pip install kaggle

# Verify Kaggle API setup
!kaggle datasets list

# Download dataset
!kaggle datasets download -d drgfreeman/rockpaperscissors

# Unzip the dataset
!unzip rockpaperscissors.zip -d rockpaperscissors

# Path to the dataset
data_dir = './rockpaperscissors'

import shutil

shutil.rmtree('./rockpaperscissors/rps-cv-images')

# Enhanced data augmentation
datagen = ImageDataGenerator(
    rescale=1.0/255,
    validation_split=0.3,
    horizontal_flip=True,
    vertical_flip=True,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    brightness_range=[0.8, 1.2],
    fill_mode='nearest'
)

# Load training and validation data
train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(200, 200),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

validation_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(200, 200),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)


print(f"Jumlah data pelatihan: {train_generator.samples}")
print(f"Jumlah data validasi: {validation_generator.samples}")

# Build a more complex model
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(200, 200, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(256, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(3, activation='softmax')
])

model.compile(
    loss='categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(),
    metrics=['accuracy']
)

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=15,  # Increase the number of epochs
    validation_data=validation_generator
)

# Save the model
model.save("rock_paper_scissors_model.keras")


Saving kaggle.json to kaggle.json
ref                                                                   title                                              size  lastUpdated          downloadCount  voteCount  usabilityRating  
--------------------------------------------------------------------  ------------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
teocalvo/teomewhy-loyalty-system                                      TeoMeWhy Loyalty System                            22MB  2024-06-13 13:00:23             54        332  1.0              
rabieelkharoua/cancer-prediction-dataset                              🩺📊 Cancer Prediction Dataset 🌟🔬                    47KB  2024-06-10 03:54:18           1423         33  1.0              
shreyanshverma27/online-sales-dataset-popular-marketplace-data        Online Sales Dataset - Popular Marketplace Data     7KB  2024-05-25 23:55:26           6578        122  1.0              
muhamm

In [ ]:
import urllib
print("Password/Enpoint IP for localtunnel is:",urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))

Password/Enpoint IP for localtunnel is: 34.125.208.254


In [ ]:
# Install Streamlit and dependencies
!pip install streamlit streamlit-webrtc opencv-python-headless pyngrok

# Write the Streamlit app to a file
with open('app.py', 'w') as f:
    f.write('''
import streamlit as st
import cv2  # Assuming OpenCV is installed for image decoding
from tensorflow import keras  # Assuming TensorFlow 2.x
import numpy as np
import random
from PIL import Image

# Load the trained model (replace with your model loading path)
model = keras.models.load_model("rock_paper_scissors_model.keras")

# Class labels
class_names = ['rock', 'paper', 'scissors']

# Generate a random action for the computer
def generate_computer_action():
    return random.choice(class_names)

# Determine the result
def determine_result(user_action, computer_action):
    if user_action == computer_action:
        return "It's a tie!"
    elif (user_action == "rock" and computer_action == "scissors") or \
         (user_action == "paper" and computer_action == "rock") or \
         (user_action == "scissors" and computer_action == "paper"):
        return "You win!"
    else:
        return "You lose!"

st.title("Rock Paper Scissors Game")
st.write("Press the button below to capture an image and play!")

captured_image = st.camera_input("Capture Image")  # Get image from webcam

if captured_image is not None:
    # Convert UploadedFile to a NumPy array and then to a PIL Image
    img = np.fromstring(captured_image.read(), np.uint8)
    img = cv2.imdecode(img, cv2.IMREAD_COLOR)  # Decode image using OpenCV
    img = Image.fromarray(img)

    # Preprocess the image
    img = img.resize((200, 200))
    img_array = np.expand_dims(np.array(img) / 255.0, axis=0)

    # Predict the user's action
    predictions = model.predict(img_array)
    user_action = class_names[np.argmax(predictions[0])]
    st.write(f"Your action: {user_action}")

    # Generate computer action
    computer_action = generate_computer_action()
    st.write(f"Computer's action: {computer_action}")

    # Determine the result
    result = determine_result(user_action, computer_action)
    st.write(result)
else:
    st.write("Waiting for image capture...")


    ''')

# Run the Streamlit app
!streamlit run app.py & npx localtunnel --port 8501


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 773.8/773.8 kB 31.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 28.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 55.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 67.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.3/34.3 MB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 82.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 37.1 MB/s eta 0:00:00



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.125.208.254:8501

npx: installed 2